## Reccomdation System
This project is Based on Content Based filtering.Content Based  filtering is used to suggest the product or things people likes.
Data set TMDB data which describe type of movies having genre,overview ,cast,crew, movie id .On the basis of similrity between two movies i am going to predict movies

In [1]:
#Importing Libraries
import pandas as pd
import numpy as np

In [2]:
#Reading Datasets
credit=pd.read_csv('tmdb_5000_credits.csv')
movies=pd.read_csv('tmdb_5000_movies.csv')

In [3]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [4]:
credit.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [5]:
#As credit datasets and movies datasets are inter realted merging both data sets on the basis of title which is present on both datasets
movies=movies.merge(credit,on='title')
##movies.info()##checking column present in Data sets and anlyasising

In [6]:
movies=movies[['genres','id','keywords','overview','title','cast','crew']] ##Removing un necessary column and keeping the column 
##which is in use

In [7]:
movies.isnull().sum()## checking Number of  null present in column
movies.dropna(inplace=True)## dropping NA values as there is only three NULL values
movies.duplicated().sum()## checking is there any duplicate value or not

0

In [8]:
movies['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [9]:
import ast
#convert function is used to fetch list of name of genre and keyword like Action and Adventure,etc
def Convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'].lower())
    return L
#Convert3 this function is used to fetch top three cast  of the movies
def Convert3(obj):
    L=[]
    count=0
    for i in ast.literal_eval(obj):
        if count==3:
            break
        L.append(i['name'].lower())
        count=count+1
    return L
# dire function is  used to fetch the name of Director  from crew features 
def dire(obj):
    L=[]
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'].lower())
    return L

In [10]:
movies['keywords']=movies['keywords'].apply(Convert)## Applying Convert function on keyword and genres
movies['genres']=movies['genres'].apply(Convert)

In [11]:
movies['cast']=movies['cast'].apply(Convert3)## Applying Convert# function on keyword and genres
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x]) 
#Removing Space Between name  so that Model can Distnguish Between Sam and Sam Billing 

In [12]:
## Breaking sentence on the basic of space
movies['overview']=movies['overview'].apply(lambda x:x.split(" "))

In [13]:
movies['crew']=movies['crew'].apply(dire)##applying dire function

In [14]:
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])
#Removing Space Between name  so that Model can Distnguish Between Sam and Sam Billing 

In [15]:
#Removing Space Between name  so that Model can Distnguish Between Sam and Sam Billing 
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [16]:
#Creating new Column Tag By adding overview,genres,keyword,cast,crew
movies['tag']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']
#Creating new Data sets which consist of id,tag,title
movies=movies[['id','title','tag']]

In [17]:
#Joining words on the basis of  space and creating  sentece or Text
movies['tag']=movies['tag'].apply(lambda x:" ".join(x))

In [18]:
# Lowering text in lower case
movies['tag']=movies['tag'].apply(lambda x:x.lower())

In [19]:
# In this section i am going to Remove English stop word, and going to convert to there Base word 
from nltk.stem import WordNetLemmatizer
import nltk
def tokenss(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    from nltk.corpus import stopwords
    lemmatizer = WordNetLemmatizer()
    # Lemmatization
    for i in range(len(sentences)):
        words = nltk.word_tokenize(sentences[i])
        words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
        sentences[i] = ' '.join(words) 
    return sentences

In [20]:
movies['tag']=movies['tag'].apply(tokenss)## applying Token functiion

In [21]:
movies['tag']=movies['tag'].apply(lambda x:" ".join(x)) ## JOINING on space

In [22]:
movies['tag'][1300]

'pair aging boxing rival coaxed retirement fight one final bout -- 30 year last match . comedy fathersonrelationship sport rivalry elderly elderlyman robertdeniro sylvesterstallone alanarkin petersegal'

In [23]:
## applying bags of words and then converting in bags of word with 5000 fetures
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)

In [24]:
vector=cv.fit_transform(movies['tag']).toarray()

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
# Us to find similarity between place n dimension place basis of angle between them
similarity=cosine_similarity(vector)

[(507, 0.2546858159868053),
 (539, 0.2537477434955704),
 (1216, 0.2491020830224699),
 (2409, 0.24507154069793594),
 (582, 0.24165612634006076)]

In [27]:
# In this section we are going to reccommond similar type of movies
def reccomd(movie):
    movie_index=movies[movies['title']==movie].index[0]
    distance=similarity[movie_index]
    mov_list=sorted(list(enumerate(distance)),reverse=True,key= lambda x:x[1])[1:6]
    for i in mov_list:
        print(movies['title'][i[0]])

In [28]:
reccomd('Batman Begins')

The Dark Knight
The Dark Knight Rises
Batman
Batman
Batman & Robin
